In [21]:
import warnings
warnings.filterwarnings('ignore') 
import pandas as pd
from sodapy import socrata
import geopandas as gdp
import contextily as ctx
import esda 
from esda.moran import Moran, Moran_Local
import splot
from splot.esda import moran_scatterplot, plot_moran, lisa_cluster, plot_moran_simulation
import libpysal as lps
import matplotlib.pyplot as plt 
import plotly.express as px
